In [3]:
# If don't have, install only once
# !pip install kagglehub

import kagglehub
import pandas as pd
import os

path = kagglehub.dataset_download("snehangsude/audible-dataset")

df = pd.read_csv(os.path.join(path, 'audible_uncleaned.csv'))

# Check first views:
# df.head()
# df.describe()
# df.info()

In [4]:
# Clean text data in Author and Narrator columns
# Remove Writtenby: from the author column
df['author'] = df['author'].str.replace('Writtenby:', '')
# Remove Narratedby: from the narrator column
df['narrator'] = df['narrator'].str.replace('Narratedby:', '')
# df.head(2)

In [5]:
# Clean stars table
# Create separate column for ratings
df['ratings'] = df['stars'].str.extract(r'stars(\d+)\s')
df['ratings'] = df['ratings'].fillna(0).astype('int')
# Create separate column for stars
df['stars out of five'] = df['stars'].str.extract(r'(\d+)\sout')
df['stars out of five'] = df['stars out of five'].fillna(0).astype('float')
df.drop('stars', axis=1, inplace=True)
# df.head(2)
# df['stars out of five'].unique()
# df['ratings'].sample(n=30)

In [6]:
# Replace the comma with ''
df['price'] = df['price'].str.replace(',', '')
# Replace 'Free' with 0
df['price'] = df['price'].str.replace('Free', '0')
# Turn price to float
df['price'] = df['price'].astype(float)
# df['price'].sample(n=30)

In [7]:
# Convert releasedate to datetime
df['releasedate'] = pd.to_datetime(df['releasedate'])
# Inspect the dataframe 
# df.info()
# Search the entries in the time column for different spellings of min. Let' try min, mins, minutes
# df['time'].str.contains(r'\b(min|mins|minutes)\b', case=False, na=False)

# Replace hrs, mins, and 'Less than 1 minute'
df['time'] = df['time'].str.replace(r'\b(mins?|minutes?)\b', 'min', case=False, regex=True)
df['time'] = df['time'].str.replace(r'\b(hr?|hrs?)\b', 'hr', case=False, regex=True)
df['time'] = df['time'].str.replace('Less than 1 minute', '1 min')
# set(type(x) for x in df['time'])
# df['time'].isna().any()

# Extract the number of hours, turn to integer
df['hours'] = df['time'].str.extract(r'(\d+)\s*hr')
df['hours'] = df['hours'].fillna("0").astype('str')
# Extract the number of minutes, turn to integer
df['mins'] = df['time'].str.extract(r'and\s(\d+)\s*min')
df['mins'] = df['mins'].fillna("0").astype('int')
df['mins'] = df['mins'].apply(lambda x: f"{x:02}")
# Combine hours and minutes into the duration column
df['duration'] = df['hours'].astype('str') + ':' + df['mins'].astype('str')

df.drop('mins', axis=1, inplace=True)
df.drop('hours', axis=1, inplace=True)
# Check the results
df.head()

/tmp/ipykernel_887/1318010996.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['releasedate'] = pd.to_datetime(df['releasedate'])


,name,author,narrator,time,releasedate,language,price,ratings,stars out of five,duration
0,Geronimo Stilton #11 & #12,GeronimoStilton,BillLobely,2 hr and 20 min,2008-04-08,English,468.0,34,5.0,2:20
1,The Burning Maze,RickRiordan,RobbieDaymond,13 hr and 8 min,2018-01-05,English,820.0,41,5.0,13:08
2,The Deep End,JeffKinney,DanRussell,2 hr and 3 min,2020-06-11,English,410.0,38,5.0,2:03
3,Daughter of the Deep,RickRiordan,SoneelaNankani,11 hr and 16 min,2021-05-10,English,615.0,12,5.0,11:16
4,"The Lightning Thief: Percy Jackson, Book 1",RickRiordan,JesseBernstein,10 hr,2010-01-13,English,820.0,181,5.0,10:00


In [8]:
# Transform prices to USD (multiply times 0.012)
df['price'] = df['price'] * 0.012
# Check the results
# df['price'].sample(n=30)

In [9]:
# Update capitalization in the language column
df['language'] = df['language'].str.capitalize()
# Check the results
# df['language'].sample(n=40)

In [10]:
# Checking the duplicates
# duplicates = df.duplicated().sum()
# print(duplicates)
subset_cols = ['name', 'author', 'narrator', 'duration', 'price']
duplicates = df.duplicated('name')
print(duplicates.sample(n=40))
# drop duplicates

#  Nan values, save to csv

36220    False
20777    False
75134    False
75780    False
1428     False
77516    False
12734    False
71161    False
19294    False
38959    False
81836    False
38002    False
36160    False
57123    False
71690    False
5242     False
40124    False
52203     True
74123    False
26360    False
18287     True
7606     False
21420    False
83692    False
59088    False
57550    False
75183    False
74010    False
2059     False
85469    False
46565    False
22274    False
22       False
74911    False
53361    False
44265    False
53096    False
28235     True
59374    False
42441    False
dtype: bool
